# Network energy

The purpose of this notebook is to reproduce two network energy models:

1. Conventional model based on average intensity from [Carbon Trust,
   2021](https://www.carbontrust.com/resources/carbon-impact-of-video-streaming)
   and with results in Table 4 on pg 52 of the report.
2. Power model from [Malmodin,
   2020](https://online.electronicsgoesgreen.org/wp-content/uploads/2020/10/Proceedings_EGG2020_v2.pdf)
   with results in Figure 5 and section 5.

Copies of both papers are provided in the `assets/` folder.

## Imports

In [56]:
%%capture
# ^ Hide the output of this cell (must be on the first line)

# Install dependencies
%pip install -r requirements.txt

## Conventional model

Carbon Trust report pg40-46.

### Global parameters

In [57]:
energyVideoStreaming = float  # Energy consumption of video streaming (E_V)
carbonVideoStreaming = float  # Carbon emissions of video streaming (C_V)
gridEmissionsFactor = 1  # ??? Emissions factor of the grid for region, n (EF_g,n)

durationStreaming = 1  # Duration of video streaming (D) Probably hours, not specified
dataTransmissionRate = (
    3  # Data transmission rate (R) GB/hr Using FHD value, not specified (pg43, Table 3)
)

### Data centers & CDNs

Carbon Trust value = 1 Wh/hour.

In [58]:
dataCenterEnergy = float  # Data center energy consumption (E_DC)
dataCenterIntensity = 1.3  # Data center intensity (I_DC) Wh/hr (pg43, Table 3)

In [59]:
# Equation (1)
dataCenterEnergy = dataCenterIntensity * durationStreaming
dataCenterEnergy

1.3

### Transmission - core & access

Carbon Trust value = 20 Wh/hour.

In [60]:
energyFixedC = float  # Fixed network energy consumption (E_FN)
intensityFixed = 0.0065  # Energy intensity of fixed network transmission (I_FN) kWh/GB (pg43, Table 3)
# proportionFixed = ? # proportion of viewing time over fixed network at data transmission rate R_i relative to the entire streaming service (P_FN,i)
energyMobileC = float  # Mobile network energy consumption (E_MN)
intensityMobile = (
    0.1  # Energy intensity of mobile network transmission (I_MN) kWh/GB (pg43, Table 3)
)
# proportionMobile = ? # Proportion of viewing time over mobile network at data transmission rate R_i relative to the entire streaming service (P_MN,i)

In [61]:
# Equation (2) - single viewing scenario
energyFixedC = intensityFixed * durationStreaming * dataTransmissionRate  # kWh
energyFixedCWh = energyFixedC * 1000  # Wh
energyFixedCWh  # Pg 52, Table 4 = 20 Wh/hour

19.5

In [62]:
# Equation (4) - single viewing scenario
energyMobileC = intensityMobile * durationStreaming * dataTransmissionRate  # kWh
energyMobileC

0.30000000000000004

### Home router (Customer Premises Equipment)

Carbon Trust value = 71 Wh/hour.

In [63]:
energyHomeRouterC = float  # Energy consumption of the home router (E_HR)
intensityHomeRouter = (
    0.025  # Energy intensity of home router transmission (I_HR) kWh/GB (pg43, Table 3)
)

In [64]:
# Equation (6) - single viewing scenario
energyHomeRouterC = (
    intensityHomeRouter * durationStreaming * dataTransmissionRate
)  # kWh
energyHomeRouterCWh = energyHomeRouterC * 1000  # Wh
energyHomeRouterCWh  # Pg 52, Table 4 = 71 Wh/hour

75.00000000000001

### End-user devices

Carbon Trust value = 10 Wh/hour (TV Peripheral) + 86 Wh/hour (screens).

In [65]:
energyDevice = float  # Energy consumption of the device (E_VD)
# powerScreens = 22 # Average power consumption of screens (W_s) W (pg 101, Table 10)
powerScreens = 86  # W (pg 52, Table 4)
# powerPeripherals = 0 # Average power consumption of peripherals (W_p) W (pg 101, Table 10)
powerPeripherals = 10  # W (pg 52, Table 4)
# proportionScreens = ? # Proportion of viewing time at W_s,i relative to the entire streaming service
# proportionPeripherals = ? # Proportion of viewing time at W_p,i relative to the entire streaming service

In [66]:
energyDevice = (powerScreens + powerPeripherals) * durationStreaming
energyDevice  # Wh/hour = 96 (pg 52, Table 4)

96

### Totals

Carbon Trust value = 188 Wh/hour.

In [67]:
# energyVideoStreamingC = dataCenterEnergy + energyFixedC + energyMobileC + energyHomeRouterC + energyDevice
energyVideoStreamingC = (
    dataCenterEnergy + energyFixedCWh + energyHomeRouterCWh + energyDevice
)
energyVideoStreamingC  # Wh/hour = 188 (pg 52, Table 4)

191.8

## Power model

The Carbon Trust report also implements this model, but does not provide any
results. Instead, it says "using the power models for fixed networks accounted
for approximately 60% of the total measured energy expected" (pg 66).

As such, the implementation here is taken from [the original paper by
Malmodin](https://online.electronicsgoesgreen.org/wp-content/uploads/2020/10/Proceedings_EGG2020_v2.pdf).

### Global parameters

In [68]:
fixedCapacity = 100  # Mbps
mobileCapacity = 10  # Mbps
netflixMbps = 4  # Mbps, from Malmodin pg 94, Figure 8

### Data centers & CDNs

Malmodin values from Table 5.

In [69]:
powerDataCenter = 7  # W

### Transmission - core

Malmodin values from section 4.5.

In [70]:
powerFixedCore = 1.5 + (0.03 * fixedCapacity)  # W + W/Mbps
powerMobileCore = 0.2 + (0.03 * mobileCapacity)  # W + W/Mbps

### Customer Premises Equipment

Malmodin values from section 5, Figure 8.

In [71]:
householdUsers = 1  # Number of users in the household
dataUsers = 1  # Number of users using the service at the same time
usedCapacity = (
    100  # % of the capacity used where 100% = 100Mbps if fixedCapacity = 100Mbps
)

powerFixedCPEIdle = 16.5 / householdUsers  # W
powerFixedCPEUse = (0.02 * (fixedCapacity / usedCapacity) * 100) / dataUsers
powerMobileCPEIdle = 1  # W
powerMobileCPEUse = 1.5 * mobileCapacity  # W + W/Mbps

### Total energy

Calculate the total energy of a 1hour Netflix streaming session in a single-user
household on a 100Mbps fixed line where the video data transmission is 4Mbps.
Malmodin value = 25 Wh (pg 94).

In [72]:
# Fixed Transmission
fixedCapacity = 100  # Mbps
netflixTransmissionPower = 1.5 + (0.03 * fixedCapacity)
netflixTransmissionPower  # W

4.5

In [73]:
# Fixed CPE
householdUsers = 1  # Number of users in the household
dataUsers = 1  # Number of users using the service at the same time
usedCapacity = (
    netflixMbps / 100
)  # % of the capacity used where 100% = 100Mbps if fixedCapacity = 100Mbps

netflixCPEPower = powerFixedCPEIdle + (0.02 * netflixMbps) / dataUsers
netflixCPEPower  # W

16.58

In [74]:
# Total power
netflixPower = netflixTransmissionPower + netflixCPEPower
netflixPower  # W

21.08

In [75]:
# Total network energy
durationStreaming = 1  # hours
netflixEnergy = netflixPower * durationStreaming
netflixEnergy  # Wh

21.08